In [65]:
import re
import csv
import math
import spacy
from spacy.symbols import nsubj, nsubjpass, dobj, iobj, pobj
from collections import Counter
from itertools import islice
from html2text import html2text
from fog.tokenizers import ngrams
from tqdm import tqdm_notebook

In [2]:
nlp = spacy.load('fr')

In [56]:
DATA = './amds_txt.csv'
STOPWORDS = './stopwords.txt'
LABELS = set([nsubj, nsubjpass, dobj, iobj, pobj])

In [4]:
WHITESPACE_RE = re.compile(r'\n+')
with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    AMENDEMENTS = [(line['id'], WHITESPACE_RE.sub(' ', html2text(line['expose']))) for line in reader if line['expose'].strip()]

In [59]:
STOPWORDS_SET = set()

with open(STOPWORDS, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        
        if line:
            STOPWORDS_SET.add(line)

In [62]:
BIGRAMS_FREQUENCIES = Counter()
TRIGRAMS_FREQUENCIES = Counter()
for i, expose in tqdm_notebook(AMENDEMENTS):
    doc = nlp(expose)

    for sentence in doc.sents:
        tokens = [token.text.lower() for token in sentence if token.is_alpha and token.text.lower() not in STOPWORDS_SET]
        
        for gram in ngrams(2, tokens):
            BIGRAMS_FREQUENCIES[tuple(gram)] += 1
            
        for gram in ngrams(3, tokens):
            TRIGRAMS_FREQUENCIES[tuple(gram)] += 1

In [63]:
BIGRAMS_FREQUENCIES.most_common()

[(('amendement', 'vise'), 4076),
 (('présent', 'amendement'), 3617),
 (('projet', 'loi'), 2502),
 (('amendement', 'propose'), 1773),
 (('article', 'code'), 1729),
 (('amendement', 'rédactionnel'), 1449),
 (('article', 'loi'), 809),
 (('mise', 'œuvre'), 805),
 (('loi', 'finances'), 783),
 (('millions', 'euros'), 730),
 (('amendement', 'objet'), 683),
 (('assemblée', 'nationale'), 657),
 (('collectivités', 'territoriales'), 640),
 (('logements', 'sociaux'), 633),
 (('mise', 'place'), 559),
 (('logement', 'social'), 510),
 (('présent', 'projet'), 496),
 (('compte', 'tenu'), 478),
 (('code', 'général'), 477),
 (('proposition', 'loi'), 476),
 (('crédit', 'impôt'), 462),
 (('article', 'projet'), 447),
 (('conseil', 'état'), 443),
 (('amendement', 'précision'), 434),
 (('présent', 'article'), 423),
 (('sécurité', 'sociale'), 409),
 (('prendre', 'compte'), 395),
 (('formation', 'professionnelle'), 371),
 (('général', 'impôts'), 366),
 (('prise', 'charge'), 353),
 (('alinéa', 'article'), 353),


In [69]:
SCORED_BIGRAMS = Counter()

for bigram, n in BIGRAMS_FREQUENCIES.items():
    score = math.log(len(BIGRAMS_FREQUENCIES) / n)
    SCORED_BIGRAMS[bigram] = score

list(reversed(SCORED_BIGRAMS.most_common()))

[(('amendement', 'vise'), 4.603380066524845),
 (('présent', 'amendement'), 4.722851228915362),
 (('projet', 'loi'), 5.091405769840604),
 (('amendement', 'propose'), 5.435823154793252),
 (('article', 'code'), 5.46095297518417),
 (('amendement', 'rédactionnel'), 5.637622518546777),
 (('article', 'loi'), 6.220452543808968),
 (('mise', 'œuvre'), 6.225409183448897),
 (('loi', 'finances'), 6.253118764876657),
 (('millions', 'euros'), 6.323206926725023),
 (('amendement', 'objet'), 6.38975660129667),
 (('assemblée', 'nationale'), 6.42856744238285),
 (('collectivités', 'territoriales'), 6.454783284513742),
 (('logements', 'sociaux'), 6.465781038723284),
 (('mise', 'place'), 6.5901019877123606),
 (('logement', 'social'), 6.6818407351490885),
 (('présent', 'projet'), 6.709675534142533),
 (('compte', 'tenu'), 6.746640728376004),
 (('code', 'général'), 6.748734969979119),
 (('proposition', 'loi'), 6.75083360663604),
 (('crédit', 'impôt'), 6.780686569785721),
 (('article', 'projet'), 6.8136928662538

In [76]:
with open('./bigrams.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['A', 'B', 'freq', 'genericity'])
    writer.writeheader()
    
    for (A, B), n in BIGRAMS_FREQUENCIES.most_common():
        
        if n < 25:
            continue
        
        writer.writerow({
            'A': A,
            'B': B,
            'freq': n,
            'genericity': math.log(len(BIGRAMS_FREQUENCIES) / n)
        })

In [64]:
TRIGRAMS_FREQUENCIES.most_common()

[(('présent', 'amendement', 'vise'), 1301),
 (('présent', 'amendement', 'propose'), 766),
 (('présent', 'projet', 'loi'), 471),
 (('article', 'projet', 'loi'), 433),
 (('code', 'général', 'impôts'), 366),
 (('projet', 'loi', 'finances'), 357),
 (('présent', 'amendement', 'objet'), 295),
 (('code', 'construction', 'habitation'), 294),
 (('amendement', 'vise', 'permettre'), 222),
 (('article', 'code', 'général'), 200),
 (('amendement', 'vise', 'préciser'), 197),
 (('amendement', 'vise', 'supprimer'), 195),
 (('objet', 'présent', 'amendement'), 184),
 (('code', 'rural', 'pêche'), 174),
 (('rural', 'pêche', 'maritime'), 170),
 (('personnes', 'situation', 'handicap'), 164),
 (('états', 'généraux', 'alimentation'), 163),
 (('article', 'code', 'construction'), 154),
 (('décret', 'conseil', 'état'), 151),
 (('amendement', 'propose', 'supprimer'), 145),
 (('article', 'loi', 'finances'), 141),
 (('projet', 'loi', 'prévoit'), 140),
 (('amendement', 'vise', 'renforcer'), 135),
 (('article', 'prése

In [14]:
FREQUENCIES_WITHOUT_AMENDEMENT = Counter()

for bigram, n in FREQUENCIES.items():
    if 'mendement' in bigram[0] or 'mendement' in bigram[1]:
        continue
        
    FREQUENCIES_WITHOUT_AMENDEMENT[bigram] += n
    
FREQUENCIES_WITHOUT_AMENDEMENT.most_common()

[(('Le', 'présent'), 2732),
 (('projet', 'loi'), 2456),
 (('article', 'code'), 1476),
 (('Il', 'agit'), 928),
 (('mise', 'œuvre'), 804),
 (('article', 'loi'), 789),
 (('loi', 'finances'), 728),
 (('millions', 'euros'), 718),
 (('logements', 'sociaux'), 633),
 (('collectivités', 'territoriales'), 631),
 (('Assemblée', 'nationale'), 613),
 (('Il', 'convient'), 568),
 (('mise', 'place'), 559),
 (('Il', 'proposé'), 554),
 (('logement', 'social'), 510),
 (('présent', 'projet'), 487),
 (('proposition', 'loi'), 469),
 (('article', 'projet'), 445),
 (('La', 'loi'), 428),
 (('crédit', 'impôt'), 427),
 (('présent', 'article'), 422),
 (('Conseil', 'État'), 420),
 (('code', 'général'), 398),
 (('prendre', 'compte'), 394),
 (('sécurité', 'sociale'), 370),
 (('formation', 'professionnelle'), 370),
 (('alinéa', 'article'), 354),
 (('prise', 'charge'), 353),
 (('demandeurs', 'asile'), 351),
 (('général', 'impôts'), 336),
 (('prise', 'compte'), 328),
 (('service', 'public'), 326),
 (('droit', 'commun')

In [54]:
NOUN_PHRASES = Counter()
for i, expose in tqdm_notebook(AMENDEMENTS):
    doc = nlp(expose)
    
    for token in doc:
        if token.dep not in LABELS or token.is_stop:
            continue
            
        noun_phrase = [t.text.lower() for t in token.subtree if not t.is_stop and t.is_alpha and len(t.text) > 3]
        
        if len(noun_phrase):
            NOUN_PHRASES[tuple(noun_phrase)] += 1

In [55]:
NOUN_PHRASES.most_common()

[(('amendement',), 5642),
 (('présent', 'amendement'), 1803),
 (('article',), 1164),
 (('elle',), 662),
 (('gouvernement',), 632),
 (('cela',), 522),
 (('projet',), 447),
 (('nous',), 442),
 (('dispositif',), 290),
 (('france',), 281),
 (('elles',), 227),
 (('cette', 'mesure'), 210),
 (('état',), 201),
 (('objectif',), 188),
 (('cette', 'disposition'), 182),
 (('mesure',), 179),
 (('rapport',), 173),
 (('objet', 'amendement'), 168),
 (('législateur',), 152),
 (('ceci',), 150),
 (('disposition',), 140),
 (('entreprises',), 140),
 (('texte',), 132),
 (('alinéa',), 130),
 (('dispositions',), 130),
 (('objectif', 'amendement'), 128),
 (('compte',), 112),
 (('cette', 'situation'), 104),
 (('personnes',), 103),
 (('transparence',), 103),
 (('présent', 'projet'), 101),
 (('administration',), 101),
 (('présent', 'article'), 97),
 (('montant',), 96),
 (('sénat',), 95),
 (('communes',), 92),
 (('derniers',), 90),
 (('objet',), 86),
 (('conseil', 'constitutionnel'), 84),
 (('conseil', 'état'), 84

In [81]:
ENTITIES = Counter()

for i, expose in tqdm_notebook(AMENDEMENTS):
    doc = nlp(expose)
    
    for entity in doc.ents:
        ENTITIES[entity.text] += 1

In [82]:
ENTITIES.most_common()

[('État', 2022),
 ('France', 1288),
 ('Gouvernement', 1046),
 ('–', 837),
 ('L.', 743),
 ('la France', 718),
 ('Sénat', 600),
 ('Parlement', 595),
 ('Assemblée nationale', 444),
 ("Conseil d'État", 411),
 ('TVA', 377),
 ('Français', 307),
 ('Etat', 307),
 ('II', 305),
 ('Constitution', 266),
 ('CSG', 262),
 ('I', 250),
 ('Paris', 243),
 ('États', 241),
 ('CITE', 220),
 ('Union européenne', 213),
 ('SNCF', 195),
 ('Conseil', 195),
 ('GOU', 176),
 ('Président de la République', 173),
 ('UE', 166),
 ('CNIL', 165),
 ('B2', 156),
 ('CFA', 154),
 ('OFPRA', 151),
 ('CPF', 151),
 ('L', 150),
 ('CESEDA', 149),
 ('APL', 148),
 ('SRU', 147),
 ('PTZ', 146),
 ('Pinel', 145),
 ('A', 135),
 ('Conseil constitutionnel', 135),
 ('Parlement européen', 133),
 ('TICPE', 133),
 ("Conseil d'Etat", 129),
 ('République', 127),
 ('PLF', 127),
 ('Rédactionnel', 126),
 ('Corse', 125),
 ('C', 122),
 ('PLU', 115),
 ('La France', 114),
 ('Europe', 111),
 ('CE', 110),
 ('CCH', 110),
 ('Commission européenne', 108),
 